In [20]:
# Import libraries
import pandas as pd
import fuzzywuzzy as fuzzywuzzy
from fuzzywuzzy import process
import datetime
from datetime import date

In [46]:
# Load up store_income_data.csv
df = pd.read_csv("store_income_data_task.csv")

In [58]:
df.columns.tolist()

['id',
 'store_name',
 'store_email',
 'department',
 'income',
 'date_measured',
 'country',
 'days_ago']

In [47]:
# Looking at the unique values in the "country" column
df.country.unique()

array(['United States/', 'Britain', ' United States', 'Britain/',
       ' United Kingdom', 'U.K.', 'SA ', 'U.K/', 'America',
       'United Kingdom', nan, 'united states', ' S.A.', 'England ', 'UK',
       'S.A./', 'ENGLAND', 'BRITAIN', 'U.K', 'U.K ', 'America/', 'SA.',
       'S.A. ', 'u.k', 'uk', ' ', 'UK.', 'England/', 'england',
       ' Britain', 'united states of america', 'UK/', 'SA/', 'SA',
       'England.', 'UNITED KINGDOM', 'America.', 'S.A..', 's.a.', ' U.K',
       ' United States of America', 'Britain ', 'England', ' SA',
       'United States of America.', 'United States of America/',
       'United States.', 's. africasouth africa', ' England',
       'United Kingdom ', 'United States of America ', ' UK',
       'united kingdom', 'AMERICA', 'America ',
       'UNITED STATES OF AMERICA', ' S. AfricaSouth Africa', 'america',
       'S. AFRICASOUTH AFRICA', 'Britain.', '/', 'United Kingdom.',
       'United States', ' America', 'UNITED STATES', 'sa',
       'United States

In [48]:
# Converting the values in the column to lowercase
df.country = df.country.str.lower()
# Removing any trailing white spaces
df.country = df.country.str.strip()

In [49]:
# The country column after string manipulation
df.country.unique()

array(['united states/', 'britain', 'united states', 'britain/',
       'united kingdom', 'u.k.', 'sa', 'u.k/', 'america', nan, 's.a.',
       'england', 'uk', 's.a./', 'u.k', 'america/', 'sa.', '', 'uk.',
       'england/', 'united states of america', 'uk/', 'sa/', 'england.',
       'america.', 's.a..', 'united states of america.',
       'united states of america/', 'united states.',
       's. africasouth africa', 'britain.', '/', 'united kingdom.',
       's. africasouth africa/', 'united kingdom/',
       's. africasouth africa.', '.'], dtype=object)

In [50]:
def replace_country_variations(df, column, string_match, matching_threshold=90):
    """
    The function that finds the matching
    between the variations found in the dataframe's column
    and the specific input string and decides to replace
    the variation with the string if the match exceeds the input threshold.

    Parameters
    ----------
    df : dataframe
        the dataframe containing the column with variations
    column : string
        the name of the column with variations
    string_match : string
        the specific string chosen to match with variations
    matching_threshold: int
        the threshold suggesting if the match found is close

    """
    unique_strings = df[column].unique()

    matches = fuzzywuzzy.process.extract(
        string_match, unique_strings, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio
    )
    print(matches)

    close_matches = [match[0] for match in matches if match[1] >= matching_threshold]

    rows_with_close_matches = df[column].isin(close_matches)

    df.loc[rows_with_close_matches, column] = string_match

    print("The replacement process with ", string_match, " is completed!")

In [51]:
# Removing all nan values and switching them with 'Other'
df.country.fillna(value="Other", inplace=True)

# Replacing the variations
replace_country_variations(df, "country", "united kingdom")
replace_country_variations(df, "country", "united states")
replace_country_variations(df, "country", "united states of america")
replace_country_variations(df, "country", "s africasouth africa")
replace_country_variations(df, "country", "britain")
replace_country_variations(df, "country", "england")
replace_country_variations(df, "country", "america")
replace_country_variations(df, "country", "uk")
replace_country_variations(df, "country", "sa")

[('united kingdom', 100), ('united kingdom.', 100), ('united kingdom/', 100), ('united states/', 52), ('united states', 52), ('united states.', 52), ('united states of america', 47), ('united states of america.', 47), ('united states of america/', 47), ('u.k.', 35)]
The replacement process with  united kingdom  is completed!
[('united states/', 100), ('united states', 100), ('united states.', 100), ('united states of america', 70), ('united states of america.', 70), ('united states of america/', 70), ('united kingdom', 52), ('britain', 30), ('britain/', 30), ('america', 30)]
The replacement process with  united states  is completed!
[('united states of america', 100), ('united states of america.', 100), ('united states of america/', 100), ('united states', 70), ('america', 45), ('america/', 45), ('america.', 45), ('united kingdom', 42), ('s. africasouth africa', 41), ('s. africasouth africa/', 41)]
The replacement process with  united states of america  is completed!
[('s. africasouth 

In [52]:
# Checking for the variations undetected by the replacement function
df.country.unique()

array(['united states', 'britain', 'united kingdom', 'u.k.', 'sa', 'u.k/',
       'america', 'Other', 's.a.', 'england', 'uk', 's.a./', 'u.k', '',
       'united states of america', 's.a..', 's africasouth africa', '/',
       '.'], dtype=object)

In [53]:
# First replacing signs like: '/', '.' with ''
# Since there are values that contain them
# And some values are equal to them
df.country.replace(regex=["\/", "\."], value="", inplace=True)
# Then finding the empty string and replacing them with 'Other'
# As they can be missing values like nan
df.country.replace(regex=[""], value="Other", inplace=True)
# Checking the unique values
df.country.unique()

array(['united states', 'britain', 'united kingdom', 'uk', 'sa',
       'america', 'Other', 'england', 'united states of america',
       's africasouth africa'], dtype=object)

In [54]:
def switching_titles_to_abbrevation(country_name):
    """
    The function which takes the name of the country
    and returns the abbrevatio of the name

    Parameters
    ----------
    country_name : string
        the string containing the name of the country

    Returns
    ----------
        abbreviation: string
            the abbreviation of the country's name
    """
    # Splitting the name by the whitespaces
    splitted_name = country_name.split(" ")
    # Taking the first letter of the first part
    first_letter_of_first_part = splitted_name[0][0]
    # Taking the first letter of the second part
    first_letter_of_second_part = splitted_name[1][0]
    # Concating the two letters
    abbreviation = first_letter_of_first_part + first_letter_of_second_part
    return abbreviation

In [55]:
# Shortening the title of the countries
# Only for the titles made of more than two words
# https://www.geeksforgeeks.org/applying-lambda-functions-to-pandas-dataframe/
df.country = df.country.apply(
    lambda x: switching_titles_to_abbrevation(x) if len(x.split(" ")) >= 2 else x
)

df.country.unique()

array(['us', 'britain', 'uk', 'sa', 'america', 'Other', 'england'],
      dtype=object)

In [56]:
# Manually matching the remaining varieties with the specific value
uk_match = "United Kingdom"
us_match = "United States"
sa_match = "South Africa"

df.country.replace("britain", uk_match, inplace=True)
df.country.replace("uk", uk_match, inplace=True)
df.country.replace("us", us_match, inplace=True)
df.country.replace("sa", sa_match, inplace=True)
df.country.replace("america", us_match, inplace=True)
df.country.replace("england", uk_match, inplace=True)

# Checking the unique values
df.country.unique()

array(['United States', 'United Kingdom', 'South Africa', 'Other'],
      dtype=object)

In [57]:
# Creating a copy of the 'date_measured' column
df["days_ago"] = df.date_measured.copy()

# Checking the type of the column
df.days_ago.dtype

dtype('O')

In [57]:
# Converting the 'days_ago' column to datetime type
df["days_ago"] = pd.to_datetime(
    df["days_ago"], format="%d %B %Y", infer_datetime_format=True
)

# Converting the today date to the compatible format
today_date = pd.to_datetime(
    datetime.date.today(), format="%d %B %Y", infer_datetime_format=True
)

# Calculating the number of days passed
# Connverting it to int since the number of days is requested
df.days_ago = (today_date - df["days_ago"]).dt.days

df.days_ago

0      19846
1      19846
2      19846
3      19846
4      19846
       ...  
995    19846
996    19846
997    19846
998    19846
999    19846
Name: days_ago, Length: 1000, dtype: int64